In [ ]:
# !pwd
# torch.cuda.is_available()

/mnt/storage/fadeev


In [1]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.1 MB/s eta 0:00:00


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
from tqdm.auto import tqdm, trange

In [7]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM

In [8]:
import gc

In [9]:
!git clone https://github.com/s-nlp/russe_detox_2022

Cloning into 'russe_detox_2022'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (82/82), 40.90 MiB | 11.31 MiB/s, done.


In [10]:
df_dev = pd.read_csv('../data/input/dev.tsv', sep='\t')
toxic_inputs_dev = df_dev['toxic_comment'].tolist()

# Model inference

In [ ]:
!pip install grpcio==1.50.0
!pip install grpcio-tools==1.50.0
!pip install protobuf==3.20

Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-4.23.1-cp37-abi3-manylinux2014_x86_64.whl (304 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.1
    Uninstalling protobuf-4.23.1:
      Successfully uninstalled protobuf-4.23.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.50.0 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 3.20.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import sentencepiece

In [13]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
model.cuda();

In [17]:
def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    inputs = tokenizer(text, return_tensors='pt')['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, 
        temperature=temperature, 
        repetition_penalty=3.0, 
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    text = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    return text

In [22]:
text = 'Дмитрий вы ебанулись? Уже все выложено'
print(paraphrase(text, model, temperature=50.0, beams=10))

['Дмитрий вы ебанулись? Уже все выложено.

— Да, уже все выложено.']


# Prompting

In [24]:
prompt = 'Сделай текст менее токсичным:'
text = 'Дмитрий вы ебанулись? Уже все выложено'
print(paraphrase(prompt + text, model, temperature=50.0, beams=10))

['Сделай текст менее токсичным:Дмитрий вы ебанулись? Уже все выложено на ютюбе.]


In [26]:
prompt = 'Удали токсичные слова из текста:'
text = 'Дмитрий вы ебанулись? Уже все выложено'
print(paraphrase(prompt + text, model, temperature=50.0, beams=10))

['Удали токсичные слова из текста:Дмитрий вы ебанулись? Уже все выложено на ютуб, а я не знаю что делать.]


In [39]:
prompt = 'Инструкция: "Удали токсичные слова из текста"'
text = 'Текст: "Дмитрий вы ебанулись? Уже все выложено"'
print(paraphrase(prompt + text, model, temperature=50.0, beams=10))

[Инструкция: Удали токсичные слова из текста Текст: Дмитрий вы ебанулись? Уже все выложено. Оригинал записи и комментарии на LiveInternet.ru]
